In [ ]:
# VaR for a portfolio

import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
import datetime as dt
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

# Equities lists tickers as symbols for crypto implementation
symbols = ['TSLA', 'AAPL', 'FB', 'AMZ']

# Weights array
weights = np.array([0.5, 0.2, 0.2, 0.1])

# Investment amount
investment_amount = 1000000

# Data
data = pdr.get_data_yahoo(symbols, start='2014-01-01', end=dt.date.today())['Close']

# Basic calculations
returns = data.pct_change()
cov_matrix = returns.cov()
avg_rtn = returns.mean()
port_mean = avg_rtn.dot(weights)
port_stdev = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
mean_investment = (1+port_mean) * investment_amount
stdev_investment = investment_amount * port_stdev 

# Confidence level and percentage point function
confidence_lvl = 0.05
cutoff = norm.ppf(confidence_lvl, mean_investment, stdev_investment)

# 1 day 95% VaR
var_1d1 = investment_amount - cutoff

# n day VaR
var_array = []
num_days = 30

for x in range(1, num_days+1):
  var_array.append(np.round(var_1d1 * np.sqrt(x), 2))
  print(str(x) + ' day VaR @ 95% confidence: ' + str(np.round(var_1d1 * np.sqrt(x), 2)))

plt.figure(figsize=(18,14))
plt.xlabel('Day #')
plt.ylabel('Max Loss ($)')
plt.title(f'Portfolio {(1 - confidence_lvl) * 100}% VaR over {num_days} days')
plt.plot(var_array)


In [ ]:
# Individual asset returns distribution histogram, against portfolio normal

plt.figure(figsize=(18,14))
returns['TSLA'].hist(bins=40, histtype='stepfilled', alpha=0.5)
x = np.linspace(port_mean - 3 * port_stdev, port_mean + 3 * port_stdev, 100)
plt.plot(x, norm.pdf(x, port_mean, port_stdev), 'r')
plt.title('AAPL returns binned vs normal distribution')
